# Question 2

# Todo List

<input type="checkbox">Create mazes of dimensions a x b with c loops, where a >= 10 and b >= 10. Save the maze in CSV format. </br>
<input type="checkbox">Place the start and goal locations anywhere inside the maze. </br>
<input type="checkbox">Implement the following search strategies to reach from the start to goal locations: </br>
<input type="checkbox">A* search using Manhattan distance </br>
<input type="checkbox">Breadth-First Search where a cell is allowed to be visited at most thrice. </br>
<input type="checkbox">Compute the amount of time that the agent needs to reach the goal location using both search strategies. </br>
<input type="checkbox">Plot visuals to compare the time that the agent needs to reach the goal using A* and breadth-first search for 5 different maze sizes. </br>
<input type="checkbox">Write a summary of observations and outcomes in a text cell within your .ipynb file. The summary should be a paragraph of no more than 100 words.

In [5]:
%pip install pyamaze

Note: you may need to restart the kernel to use updated packages.


# Breadth-first Search

In [1]:
from collections import deque
import time

def BFS(m, goal):
    # Initialization
    start = (m.rows, m.cols)
    explored = set([start]) # utilizing set for better performance compared to list 
    frontier = deque([start]) 

    # print("original frontier=", frontier)
    # print("original explored=", explored)

    bfsPath = {}
    searchPath = []
    visitedCount = {}  # Counter to track visited cells

    start_time = time.time()

    # Continue loop as long as frontier is not empty
    while frontier:
        # Pops based on FIFO
        currCell = frontier.popleft()
        searchPath.append(currCell)

        # If current cell == goal, then end
        if currCell == (goal[0], goal[1]):
            break

        # Look into all 4 directions
        for d in "ESNW":
            if m.maze_map[currCell][d]:
                if d == "E":
                    childCell = (currCell[0], currCell[1] + 1)
                elif d == "W":
                    childCell = (currCell[0], currCell[1] - 1)
                elif d == "S":
                    childCell = (currCell[0] + 1, currCell[1])
                elif d == "N":
                    childCell = (currCell[0] - 1, currCell[1])

                if childCell in explored:
                    # Check if the cell has been visited less than 3 times
                    if visitedCount.get(childCell, 0) < 3:
                        visitedCount[childCell] = visitedCount.get(childCell, 0) + 1
                        frontier.append(childCell)
                    continue

                explored.add(childCell)
                frontier.append(childCell)

                # print("frontier=", frontier)
                # print("explored=", explored)

                bfsPath[childCell] = currCell

    end_time = time.time()
    elapsed_time = end_time - start_time
    
    text = "\nBFS Path\n"
    text += "\n=============================================\n"
    # Print out content of bfsPath
    text += "{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in bfsPath.items()) + "}"

    fwdPath = {}
    cell = (goal[0], goal[1])
    text = "\n=============================================\n"

    while cell != start:
        fwdPath[bfsPath[cell]] = cell
        cell = bfsPath[cell]

    # text += "\nBFS Fwd Path\n"
    # text += "\n=============================================\n"
    # text += "{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in fwdPath.items()) + "}"
    
    filename = "BFS Output.txt"
    with open(filename, 'w') as file:
        file.write(text)

    return searchPath,bfsPath,fwdPath, elapsed_time


# A* Search

In [2]:
from pyamaze import maze,agent,COLOR,textLabel
from queue import PriorityQueue
import time

def h(cell1,cell2):
    x1,y1=cell1
    x2,y2=cell2
    return abs(x1-x2) + abs(y1-y2) 

def aStar(m,goal):
    start=(m.rows,m.cols)
    g_score={cell:float('inf') for cell in m.grid}
    g_score[start]=0
    f_score={cell:float('inf') for cell in m.grid}
    f_score[start]=h(start,(goal[0],goal[1]))

    open_set=PriorityQueue()
    open_set.put((h(start,(goal[0],goal[1])),h(start,(goal[0],goal[1])),start))
    aPath={}
    
    searchPath = []
    start_time = time.time()
    while not open_set.empty():
        currCell = open_set.get()[2]
        searchPath.append(currCell)
        
        if currCell == (goal[0],goal[1]):
            break
        for d in 'ESNW':
            if m.maze_map[currCell][d] == True:
                if d == 'E':
                    childCell = (currCell[0],currCell[1]+1)
                if d == 'S':
                    childCell = (currCell[0]+1,currCell[1])
                if d == 'N':
                    childCell = (currCell[0]-1,currCell[1])
                if d == 'W':
                    childCell = (currCell[0],currCell[1]-1)
                    
                temp_g_score = g_score[currCell] + 1
                temp_f_score = temp_g_score+h(childCell,(goal[0],goal[1]))
                
                if temp_f_score < f_score[childCell]:
                    g_score[childCell] = temp_g_score
                    f_score[childCell] = temp_f_score
                    open_set.put((temp_f_score,h(childCell,(goal[0],goal[1])),childCell))
                    aPath[childCell] = currCell    
    fwdPath={}
    cell=(goal[0],goal[1])
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    
    
    while cell!=start:
        fwdPath[aPath[cell]]=cell
        cell=aPath[cell]
        
    text = "\nA* Search Path\n"
    text += "\n=============================================\n"
    text += "{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in aPath.items()) + "}"
    
    filename = "A* Search Output.txt"
    i = 0
    while (open(filename, 'w') == True) :
        filename = "A* Search Output.txt" + str(i)
    with open(filename, 'w') as file:
        file.write(text)
    
    return fwdPath,aPath,searchPath,elapsed_time

# Main Driver Function

## Maze creation

In [3]:
import random
from pyamaze import maze, agent, COLOR

a = 10 #size of maze
b = 10 #size of maze
c = 10 #loop percent

# get random value for goal
goal = random.randint(1,a - 1),random.randint(1,b - 1)

m = maze(a,b) #instantiation of maze
m.CreateMaze(goal[0],goal[1],loopPercent=c,saveMaze=True) #saveMaze exports the maze into a csv file

# BFS Search
bfsPath=BFS(m,goal)
bfsSearchAgent=agent(m,footprints=True,color=COLOR.yellow,filled=True)
bfsPathAgent=agent(m,footprints=True)
m.tracePath({bfsSearchAgent:bfsPath[0]},showMarked=True)
m.tracePath({bfsPathAgent:bfsPath[2]})

# A* Search
aPath=aStar(m,goal) 
aSearchAgent=agent(m,footprints=True,color=COLOR.red,filled=True)
aPathAgent=agent(m,footprints=True)
m.tracePath({aSearchAgent:aPath[2]},showMarked=True)
m.tracePath({aPathAgent:aPath[0]})

# description = 	"""BFS Search : 
# 	Search Path Colour : Yellow
# 	Elapsed Computation Time : """ + str(bfsPath[3]) + """ seconds
# A* Search :
# 	Search Path Colour : Red
# 	Elapsed Computation Time : """ + str(aPath[3]) + """ seconds
# 	"""
# label_position = (a + 1, b + 1)  # The position of the label (top right corner)

# label = textLabel(m, description, label_position)
m.run()

: 

: 

# Loading from an already existing maze

In [ ]:
import random
from pyamaze import maze, agent, COLOR

a = 10 #size of maze
b = 10 #size of maze
c = 10 #loop percent

# get random value for goal

mazeName = 'maze20x20.csv'
m = maze() #instantiation of maze
goal = 1,1
m.CreateMaze(loadMaze=mazeName) #saveMaze exports the maze into a csv file

# BFS Search
bfsPath=BFS(m,goal)
bfsSearchAgent=agent(m,footprints=True,color=COLOR.yellow,filled=True)
bfsPathAgent=agent(m,footprints=True)
m.tracePath({bfsSearchAgent:bfsPath[0]},showMarked=True)
m.tracePath({bfsPathAgent:bfsPath[2]})

# A* Search
aPath=aStar(m,goal) 
aSearchAgent=agent(m,footprints=True,color=COLOR.red,filled=True)
aPathAgent=agent(m,footprints=True)
m.tracePath({aSearchAgent:aPath[2]},showMarked=True)
m.tracePath({aPathAgent:aPath[0]})

# description = 	"""BFS Search : 
# 	Search Path Colour : Yellow
# 	Elapsed Computation Time : """ + str(bfsPath[3]) + """ seconds
# A* Search :
# 	Search Path Colour : Red
# 	Elapsed Computation Time : """ + str(aPath[3]) + """ seconds
# 	"""
# label_position = (a + 1, b + 1)  # The position of the label (top right corner)

# label = textLabel(m, description, label_position)
m.run()

FileNotFoundError: [Errno 2] No such file or directory: 'maze20x20.csv'